In [1]:
import syft as sy
import torch as th
from syft.core.plan.plan_builder import make_plan

In [2]:
alice = sy.VirtualMachine()
alice_client = alice.get_root_client()

[2021-03-23T11:27:16.543157+0800][CRITICAL][logger]][1314] Skipping torchvision.torchvision.transforms.functional.adjust_sharpness not supported in 0.8.1
[2021-03-23T11:27:16.543519+0800][CRITICAL][logger]][1314] Skipping torchvision.torchvision.transforms.functional.autocontrast not supported in 0.8.1
[2021-03-23T11:27:16.543763+0800][CRITICAL][logger]][1314] Skipping torchvision.torchvision.transforms.functional.equalize not supported in 0.8.1
[2021-03-23T11:27:16.544016+0800][CRITICAL][logger]][1314] Skipping torchvision.torchvision.transforms.functional.invert not supported in 0.8.1


In [3]:
# a layer
fc = th.nn.Linear(25,10)
fc_ptr = fc.send(alice_client)
print(fc_ptr)
res_ptr = fc_ptr(th.rand([1,25]))
print(res_ptr)
print(res_ptr.get())
print(fc_ptr.get())

[2021-03-23T11:27:16.544251+0800][CRITICAL][logger]][1314] Skipping torchvision.torchvision.transforms.functional.posterize not supported in 0.8.1
[2021-03-23T11:27:16.544511+0800][CRITICAL][logger]][1314] Skipping torchvision.torchvision.transforms.functional.solarize not supported in 0.8.1


tensor([[ 0.3970, -0.0868, -0.7340, -0.3398, -0.4073,  0.3373, -0.7512,  0.3379,
         -0.5889,  0.4390]], requires_grad=True)
Linear(in_features=25, out_features=10, bias=True)


/mnt/d/github/PySyft/src/syft/lib/torch/uppercase_tensor.py:30: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(obj, "grad", None)


In [4]:
# container of layers
seq = th.nn.Sequential()
seq.add_module("fc1", th.nn.Linear(25,10))
seq.add_module("fc2", th.nn.Linear(10,1))
seq_ptr = seq.send(alice_client)
print(seq_ptr)
res_ptr = seq_ptr(th.rand([1,25]))
print(res_ptr)
print(res_ptr.get())
print(seq_ptr.get())

[2021-03-23T11:27:16.669363+0800][CRITICAL][logger]][1314] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: cb366eea6f16403da2b067c5b7bbb328>.


tensor([[-0.2357]], requires_grad=True)
Sequential(
  (fc1): Linear(in_features=25, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)


In [5]:
# user defined model
class M(th.nn.Module):
    def __init__(self):
        super(M, self).__init__()
        self.fc1 = th.nn.Linear(25,10)
        self.fc2 = th.nn.Linear(10,1)
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
    @property
    def forward_Plan(self):
        @make_plan
        def remote_forward(
            x=th.rand([4,25]), 
            fc1 = self.fc1,
            fc2 = self.fc2
        ):
            x = fc1(x)
            x = fc2(x)
            return x
        return remote_forward
        
m = M()
m_ptr = m.send(alice_client)
print(m_ptr)
# res_ptr = mptr(th.rand([1,25])) # TODO
# print(res_ptr) # TODO
# print(res_ptr.get()) # TODO
print(m_ptr.get())

Module(
  (fc1): Linear(in_features=25, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)
